# Adding Sucrose Export Reactions to Synechococcus elongatus pcc 7942

In [1]:
import cobra
import gurobipy
import pandas as pd
import os

In [2]:
os.environ["GRB_LICENSE_FILE"]

'/Users/mcna892/Desktop/Projects/Digital_Twins/gurobi.lic'

## Load in the model and universal model

In [3]:
model_syn = cobra.io.read_sbml_model('syn_elong.xml')
universal_full = cobra.io.load_json_model('universal_model_cobrapy.json')

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


## Find Sucrose Extraction Reactions

From the universal model we can find reactions that are involved in the transport of Sucrose `sucr` from the extracellular to cellular environment

In [6]:
universal_full.metabolites.sucr_e

Metabolite identifier,sucr_e
Name,Sucrose C12H22O11
Memory address,0x2838c1180
Formula,None
Compartment,e
In 6 reaction(s),"SUCRt2, SUCpts, EX_sucr_e, SUCRtex, SUCRe, SUCRabc"


From here we can select one that represents known genetic additions. In this case we want to look at the `SUCRt2` reaction. It can be found as an addition made to the organism to export sucrose from Synechococcus [[citation](https://doi.org/10.1128/AEM.07901-11)]. We also want to ensure that we add the corresponding exchange reaction `EX_sucr_e` to the model so the FBA methods can run.

In [4]:
sucr_exchange = universal_full.reactions.EX_sucr_e.copy()
sucr_transport = universal_full.reactions.SUCRt2.copy()

If we look at the transport we can see that it only converts extracellular sucrose `sucr_e` to cellular sucrose `sucr_e` but not the other way around.

In [5]:
sucr_transport

Reaction identifier,SUCRt2
Name,Sucrose transport in via proton symport
Memory address,0x28c25e3e0
Stoichiometry,h_e + sucr_e --> h_c + sucr_c H+ + Sucrose C12H22O11 --> H+ + Sucrose C12H22O11
GPR,
Lower bound,0.0
Upper bound,1000.0


We know from the above citation that the added proton symport (cscB) does indeed serve to export `sucr_c` back to `sucr_e`, therefore we can edit the lower bound to create a reversible reaction.

In [8]:
sucr_transport.lower_bound = -1000.0
sucr_transport

Reaction identifier,SUCRt2
Name,Sucrose transport in via proton symport
Memory address,0x28c25e3e0
Stoichiometry,h_e + sucr_e <=> h_c + sucr_c H+ + Sucrose C12H22O11 <=> H+ + Sucrose C12H22O11
GPR,
Lower bound,-1000.0
Upper bound,1000.0


Now the reaction is reveresible and we can add these reactions to `model_syn`.

In [9]:
model_syn.add_reaction(sucr_exchange)
model_syn.add_reaction(sucr_transport)

Let's check the reactions to make sure they are added.

In [10]:
model_syn.reactions.SUCRt2

Reaction identifier,SUCRt2
Name,Sucrose transport in via proton symport
Memory address,0x28c25e3e0
Stoichiometry,h_e + sucr_e <=> h_c + sucr_c H+ + Sucrose C12H22O11 <=> H+ + Sucrose C12H22O11
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [11]:
model_syn.reactions.EX_sucr_e

Reaction identifier,EX_sucr_e
Name,Sucrose exchange
Memory address,0x28c25e3b0
Stoichiometry,sucr_e --> Sucrose C12H22O11 -->
GPR,
Lower bound,0.0
Upper bound,1000.0


## Checking if Sucrose is Exported

Now that we have added the reactions, we can optimize it and check to see if it now secretes __Sucrose__.

In [14]:
model_syn.summary().secretion_flux.loc['EX_sucr_e']

flux                0.0
reaction      EX_sucr_e
metabolite       sucr_e
Name: EX_sucr_e, dtype: object

Here we can see that there is 0 flux going through the model during optimization. Let us see what happens when we shift the objective function to optimze specifically the Exchange reaction.

In [16]:
model_syn.objective = 'EX_sucr_e'
model_syn.summary().secretion_flux.loc['EX_sucr_e']

flux          -0.331667
reaction      EX_sucr_e
metabolite       sucr_e
Name: EX_sucr_e, dtype: object

Great! Now we can see that sucrose is being secreted from the cell. Let's look at the full summary.

In [18]:
model_syn.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,1.99,1,50.00%
h2o_e,EX_h2o_e,1.659,0,0.00%
h_e,EX_h_e,1.988,0,0.00%
hco3_e,EX_hco3_e,1.99,1,50.00%
photon670_e,EX_photon670_e,14.77,0,0.00%
photon690_e,EX_photon690_e,22.56,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
pho_loss_c,DM_pho_loss_c,-0.7384,0,0.00%
o2_e,EX_o2_e,-3.98,0,0.00%
sucr_e,EX_sucr_e,-0.3317,0,0.00%


Based on experimental evidence, we know that sucrose should be excreted from the cell. Since it is not under normal optimization but all the pathways are connected when optimizing the exchange directly, we can simply enforce a minimum flux by setting the reactions lower bound.

In [20]:
model_syn.reactions.EX_sucr_e.lower_bound = 0.011

Now that we have set the lower bound, we can change the objective back to the `BIOMASS_1` reaction and check the summary again.

In [22]:
model_syn.objective = 'BIOMASS__1'
model_syn.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,0.73,1,29.48%
cobalt2_e,EX_cobalt2_e,2.758E-07,0,0.00%
fe3_e,EX_fe3_e,8.635E-05,0,0.00%
h_e,EX_h_e,2.003,0,0.00%
hco3_e,EX_hco3_e,1.576,1,63.65%
leu__L_e,EX_leu__L_e,0.02836,6,6.87%
mg2_e,EX_mg2_e,0.0006043,0,0.00%
nh4_e,EX_nh4_e,0.0031,0,0.00%
no3_e,EX_no3_e,0.4029,0,0.00%
photon630_e,EX_photon630_e,12.19,0,0.00%


We now have `sucr_e` under the sectretion fluxes while optimizing for the `BIOMASS__1` reaction. This means that the model is growing as well as exporting sucrose under normal conditions.

## Setting the Names of the Genes

Now that we have added 2 additional reactions, we can assign them gene names. Since `EX_sucr_e` is not associated with a gene, we do not need to add one. For `SUCRt2`, we know that this is associated with the E. Coli gene `cscB`. We can go out and pull relevant information regarding this from the literature. We can find that the gene id is listed as `ATCC 700927`. Therefore we can create a cobra gene object to add to the model.

In [41]:
gene_add = cobra.core.Gene(id='cscB',name='cscB',functional=True)
gene_add

Gene identifier,ATCC_700927
Name,ATCC_700927
Memory address,0x28628c220
Functional,True
In 0 reaction(s),


Now that we have `gene_add`, we can associate the gene with the transport reaction that we added before.

In [42]:
model_syn.reactions.SUCRt2.gene_reaction_rule = '( ATCC_700927 )'

Now we can check to see if it has been added.

In [47]:
model_syn.genes.ATCC_700927

Gene identifier,ATCC_700927
Name,
Memory address,0x2857eb4c0
Functional,True
In 1 reaction(s),SUCRt2


## Saving the New Model

Now that we have an edited model, we can save our model as new iteration of the model to pass back into our Version Control System (MEMOTE).

In [49]:
#cobra.io.write_sbml_model(model_syn, 'syn_w_cscB.xml')